학습과정을 Notion에 추가 정리

In [1]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.4 MB/s eta 0:00:00


In [22]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=16e384a04df3ed43391ea514842339bce4a32c721c5323911bcc19f43089d4d3
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
# model compile / layer 쌓기
def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(256, 3, activation='relu'),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(256, activation='relu'),
       tf.keras.layers.Dense(128, activation='relu'),
       tf.keras.layers.Dense(10)])

In [23]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, shape_list, BertTokenizerFast
from transformers import TFBertForSequenceClassification, TextClassificationPipeline
from sklearn import preprocessing
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report

In [ ]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7b3589eb2f50>)

In [ ]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

# data 확인
print('train review :',len(train_data)) # 훈련용 리뷰 개수
print('test review :',len(test_data)) # 테스트용 리뷰 개수

train review : 150000
test review : 50000


In [ ]:
# 중복과 null값 제거
train_data.drop_duplicates(subset=['document'], inplace=True)
train_data = train_data.dropna(how='any')

test_data = test_data.dropna(how = 'any')

print('train review:',len(train_data))
print('test review:',len(test_data))

train review: 146182
test review: 49997


### Input of BERT
BERT가 필요한 입력
- 문장 구분을 위한 세그먼트 인코딩
- Word embedding
- 단어 token과 padding token을 구분하기 위한 Attention Mask

In [ ]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [ ]:
# tokenizer 성능 확인해보기 - 토큰화
print(tokenizer.tokenize("보는 내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['보', '##는', '내내', '그대로', '들어맞', '##는', '예측', '카리스마', '없', '##는', '악역']


In [ ]:
# 정수 인코딩
print(tokenizer.encode("보는 내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[2, 1160, 2259, 6404, 4311, 20657, 2259, 5501, 13132, 1415, 2259, 23713, 3]


In [ ]:
# 디코딩하여 원래 문장이 어떻게 되었는지 확인하기 -> [CLS] / [SEP] 확인하기
# 2, 3은 기존 단어가 아닌 특수 토큰
tokenizer.decode(tokenizer.encode("보는 내내 그대로 들어맞는 예측 카리스마 없는 악역"))

'[CLS] 보는 내내 그대로 들어맞는 예측 카리스마 없는 악역 [SEP]'

In [ ]:
# 실제 특수 token id 확인해보기
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':' , tokenizer.sep_token_id)
print(tokenizer.pad_token, ':', tokenizer.pad_token_id)

[CLS] : 2
[SEP] : 3
[PAD] : 0


FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` \
책에서 제시한 pad_to_max_length 보단 위와 같은 방법을 padding을 위해 사용할 것

In [ ]:
# 최대 길이 128로 제한
max_seq_len = 128

# 정수 인코딩 및 최대 길이까지 padding 수행
encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시보고 싶은 영화", \
                                  max_length=max_seq_len, padding='max_length')
print(encoded_result)
print('length :', len(encoded_result))

[2, 1537, 2534, 2069, 6572, 2259, 3771, 18, 3690, 2178, 2088, 1335, 2073, 3771, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
length : 128


In [ ]:
# 세그먼트 인코딩
# 하나의 문장만 존재하기 때문에 그냥 문장 길이 만큼의 0 벡터를 생성한다.
print([0]*max_seq_len)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# Attention Mask 만들기
valid_num = len(tokenizer.encode("전율을 일으키는 영화. 다시보고 싶은 영화"))

# 실제 단어가 있는 곳은 1로 표시, 나머지 Max_len에서 실제 단어를 제외한 곳은 0으로 채우기
print(valid_num * [1] + (max_seq_len - valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# 위 과정을 요약한 function
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
  input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

  for example, label in tqdm(zip(examples, labels), total=len(examples)):

    # input_id: Word embedding을 위한 정수 인코딩
    input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)

    # attention_mask: 실제 token이 있다면 1, padding이라면 0
    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

    # token_type_id: 문장 구분을 위한 세그먼트 인코딩(여기서는 문장이 하나니까 문장 길이만큼의 0 벡터)
    token_type_id = [0] * max_seq_len

    assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

    # 생성한 벡터들을 리스트에 추가
    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    data_labels.append(label)

  # Numpy 배열로 생성
  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)

  # copy까지 등장하는 모든 data_label에 해당하는 값을 일괄변경
  data_labels = np.asarray(data_labels, dtype=np.int32)

  return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
# data에 적용
train_X, train_y = convert_examples_to_features(train_data['document'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)
test_X, test_y = convert_examples_to_features(test_data['document'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 49997/49997 [00:35<00:00, 1418.77it/s]


In [ ]:
# 결과 확인해보기
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]
print('단어에 대한 정수 인코딩 :',input_id)
print('Attention Mask :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('Decoding :',tokenizer.decode(input_id))
print('Label :',label)

단어에 대한 정수 인코딩 : [   2 1376  831 2604   18   18 4229 9801 2075 2203 2182 4243    3    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
Attention Mask : [1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
세그먼트 인코딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

### Output of BERT

In [ ]:
model = TFBertModel.from_pretrained("klue/bert-base", from_pt=True)

max_seq_len = 128

input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'bert.embeddings.position_ids', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
# 문장 길이만큼의 출력
# output = (배치 크기, 128, 768)크기의 텐서
# Many‑to‑Many

print(outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 128, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0', description="created by layer 'tf_bert_model'")


In [ ]:
# CLS 토큰 위치 출력
# Many‑To‑One
print(outputs[1])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/pooler/dense/Tanh:0', description="created by layer 'tf_bert_model'")


## Mant-to-One Model by BERT - 네이버 영화 리뷰 분류하기

### Data Encoding 결과 확인해보기

In [ ]:
# 토큰나이저 호출
tokenizer = BertTokenizerFast.from_pretrained("klue/bert-base")

# 리뷰와 레이블 데이터 분리하기
X_train_list = train_data['document'].tolist()
X_test_list = test_data['document'].tolist()
y_train = train_data['label'].tolist()
y_test = test_data['label'].tolist()

In [ ]:
# 자동으로 세그먼트와 padding 수행
# truncation: max_length가 지정되지 않은 경우 모델이 허용하는 최대 길이로 문장을 자른다.
X_train = tokenizer(X_train_list, truncation=True, padding=True)
X_test = tokenizer(X_test_list, truncation=True, padding=True)

In [ ]:
# 결과 확인하기
# token: 토큰화 결과
# ids: 정수 인코딩 결과
# type_ids: 세그먼트 인코딩 결과
# attention_mask: Attention Mask 확인 가능

print(X_train[0])

Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


###  데이터셋 생성 및 모델 학습

In [ ]:
# tf.data.Dataset: 텐서플로에 맞는 데이터 형식으로 바꾸어주는 Class
train_dataset = tf.data.Dataset.from_tensor_slices((dict(X_train), y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(X_test), y_test))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

In [ ]:
# num_labels = 구분하고자 하는 class의 수
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base",num_labels=2, from_pt=True)

# 손실함수: cross entropy
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# early stopping 적용해보기
early_stopping = EarlyStopping(monitor="val_accuracy",
                               min_delta=0.001,
                               patience=2)

# 모델 학습
model.fit(train_dataset.shuffle(10000).batch(32), epochs=2, batch_size=32,
          validation_data = val_dataset.shuffle(10000).batch(32),
          callbacks = [early_stopping])

Epoch 1/2
4569/4569 [==============================] - 4227s 925ms/step - loss: 0.2270 - accuracy: 0.9077 - val_loss: 0.2575 - val_accuracy: 0.8931
Epoch 2/2
4569/4569 [==============================] - 4226s 925ms/step - loss: 0.1624 - accuracy: 0.9377 - val_loss: 0.2682 - val_accuracy: 0.8979


In [ ]:
# 내 드라이브에 모델 저장하기
model.save_pretrained('/content/drive/MyDrive/nsmc_model/bert-base')
tokenizer.save_pretrained('/content/drive/MyDrive/nsmc_model/bert-base')

('/content/drive/MyDrive/nsmc_model/bert-base/tokenizer_config.json',
 '/content/drive/MyDrive/nsmc_model/bert-base/special_tokens_map.json',
 '/content/drive/MyDrive/nsmc_model/bert-base/vocab.txt',
 '/content/drive/MyDrive/nsmc_model/bert-base/added_tokens.json',
 '/content/drive/MyDrive/nsmc_model/bert-base/tokenizer.json')

### TEST
TextClassificationPipeline 은 모델의 예측값이 소프트맥스 함수를 통과한 후의 값인 레이블 별 스코어를 보여준다

In [ ]:
# 저장한 tokenizer와 model불러오기 -> 경로 입력 제대로 하기
loaded_tokenizer = BertTokenizerFast.from_pretrained('/content/drive/MyDrive/nsmc_model/bert-base')
loaded_model = TFBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/nsmc_model/bert-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some layers from the model checkpoint at /content/drive/MyDrive/nsmc_model/bert-base were not used when initializing TFBertForSequenceClassification: ['dropout_74']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/nsmc_model/bert-base.
If your task is similar to the task the model of the checkpoint was trained on,

In [ ]:
# 테스트기 만들기
text_classifier = TextClassificationPipeline(tokenizer=loaded_tokenizer,
                                             model=loaded_model,
                                             framework='tf',
                                             return_all_scores=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# random sentence 넣기
text_classifier('뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아')[0]

[{'label': 'LABEL_0', 'score': 0.8199568390846252},
 {'label': 'LABEL_1', 'score': 0.18004316091537476}]

## KoBERT 를 이용한 KorNLI 풀어보기 (다중 클래스 분류)

### 데이터 로드하기

In [3]:
# train data
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", \
                           filename="multinli.train.ko.tsv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", \
                           filename="snli_1.0_train.ko.tsv")

# validation data
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", \
                           filename="xnli.dev.ko.tsv")
# test data
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", \
                           filename="xnli.test.ko.tsv")

('xnli.test.ko.tsv', <http.client.HTTPMessage at 0x78a70e2212d0>)

In [4]:
train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

In [12]:
# 2개의 학습 데이터 결합 후 섞기
train_data = train_snli.append(train_xnli)
train_data = train_data.sample(frac=1)

<ipython-input-12-2f3c223789b6>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_snli.append(train_xnli)


In [13]:
train_data.head()

,sentence1,sentence2,gold_label
70265,회색 탱크톱을 입은 금발 소녀가 앉아 있는 흰색 셔츠를 입은 남자에게 술을 대접하고...,탱크톱에 있는 금발 소녀가 남자를 대접하고 있다.,entailment
72962,사람들이 풀밭에 앉아 있는 동안 한 어린 아이가 커다란 빨간 모자를 쓰고 길 한복판...,어린 아이가 집 안에 앉아 있다.,contradiction
231540,여주인이 다투었다? &quot;,그녀는 비밀스러운 삶을 살았다.,neutral
69401,스틱의 검은 눈이 굴렀고 그는 무더기에 빠졌다.,그는 나무 바닥의 젖은 부분에 미끄러져 쓰러졌다.,contradiction
254604,그는 달걀을 파는 것에 대해 진지하지 않다고 포스트는 말한다. 그는 단지 그의 모델...,"그는 인간 난자를 홍보용 묘기로 사용하고 있는데, 그것은 슬프다.",neutral


In [14]:
# 중복/결측치 제거 함수
def drop_na_and_duplciates(df):
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop=True)
  return df

In [15]:
train_data = drop_na_and_duplciates(train_data)
val_data = drop_na_and_duplciates(val_data)
test_data = drop_na_and_duplciates(test_data)

### Input of BERT

In [16]:
# tokenizer 호출
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

# 최대길이 = 128
max_seq_len = 128

sent1 = train_data['sentence1'].iloc[0]
sent2 = train_data['sentence2'].iloc[0]

print('문장1 :',sent1)
print('문장2 :',sent2)

문장1 : 회색 탱크톱을 입은 금발 소녀가 앉아 있는 흰색 셔츠를 입은 남자에게 술을 대접하고 있다.
문장2 : 탱크톱에 있는 금발 소녀가 남자를 대접하고 있다.


In [17]:
# encode_plus: 정수 인코딩, 세그먼트 임베딩, attention mask 생성 및 return
encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=max_seq_len,pad_to_max_length=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [18]:
# 임베딩 벡터들 확인해보기
print(encoding_result)

{'input_ids': [2, 13203, 11579, 2734, 2069, 1511, 2073, 21459, 5736, 2116, 1379, 2227, 1513, 2259, 12003, 10727, 2138, 1511, 2073, 3997, 2170, 2318, 1299, 2069, 9087, 19521, 1513, 2062, 18, 3, 11579, 2734, 2170, 1513, 2259, 21459, 5736, 2116, 3997, 2138, 9087, 19521, 1513, 2062, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [19]:
# 위 과정을 함수화
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):
  input_ids, attention_masks, token_type_ids = [], [], []

  # encode_plus를 통해 정수 인코딩, 세그먼트 인코딩, attention mask를 생성
  for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
    encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=max_seq_len, pad_to_max_length=True)

    # 생성된 임베딩 벡터들을 리스트에 append
    input_ids.append(encoding_result['input_ids'])
    attention_masks.append(encoding_result['attention_mask'])
    token_type_ids.append(encoding_result['token_type_ids'])

  # 리스트를 모두 numpy array로 변환
  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)

  return (input_ids, attention_masks, token_type_ids)

이 아래부터는 모델 돌리는 시간도 오래 걸리고 data 전처리를 수행하면서 문제가 있는 것 같이 실행하지 않았음\
코드 리뷰에 집중

In [ ]:
# 전처리하는 방식
# 이후 test와 validation을 위한 data에도 동일하게 수행한다.
X_train = convert_examples_to_features(train_data['sentence1'], train_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)
X_test = convert_examples_to_features(test_data['sentence1'], test_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

train_label = train_data['gold_label'].tolist()
val_label = val_data['gold_label'].tolist()
test_label = test_data['gold_label'].tolist()

idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

# 고유한 정수로 변환
y_train = idx_encode.transform(train_label)
y_val = idx_encode.transform(val_label)
y_test = idx_encode.transform(test_label)
label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print('각 레이블과 정수:', label_idx)

In [ ]:
# 모델 생성 - 다중 클래스 분류
class TFBertForSequenceClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForSequenceClassification, self).__init__()

    # 사전학습된 bert를 가져옴
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)

    # 분류기 생성 - 분류하고 하는 class의 개수만큼
    self.classifier = tf.keras.layers.Dense(num_labels,
                                            kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),activation='softmax',\
                                            name='classifier')

    # 입력으로 임베딩 벡터들을 받음
    def call(self, inputs):
      input_ids, attention_mask, token_type_ids = inputs

      # bert에 임베딩 벡터를 넣음
      outputs = self.bert(input_ids=input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)

      cls_token = outputs[1]
      prediction = self.classifier(cls_token)

      return prediction

In [ ]:
# early stopping 적용
# min_delta: 개선된 것으로 판단하기 위한 최소한의 변화량
early_stopping = EarlyStopping(monitor="val_accuracy",
                               min_delta=0.001,
                               patience=2)
model.fit(
X_train, y_train, epochs=2, batch_size=32, validation_data = (X_val, y_val), callbacks = [early_stopping])

In [ ]:
# 만약 hugging face에서 제공하는 TFBertForSequenceClassification을 사용한다면? (추천)
import tensorflow as tf
from transformers import TFBertForSequenceClassification

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base",
num_labels=3, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

## KLUE‑BERT 를 이용한 개체명 인식

In [25]:
# data download
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_train_data.csv",
                           filename="ner_train_data.csv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_test_data.csv",
                           filename="ner_test_data.csv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_label.txt",
                           filename="ner_label.txt")

('ner_label.txt', <http.client.HTTPMessage at 0x78a5ca3026b0>)

In [26]:
train_ner_df = pd.read_csv("ner_train_data.csv")
test_ner_df = pd.read_csv("ner_test_data.csv")

In [27]:
train_data_sentence = [sent.split() for sent in train_ner_df['Sentence'].values]
test_data_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]
train_data_label = [tag.split() for tag in train_ner_df['Tag'].values]
test_data_label = [tag.split() for tag in test_ner_df['Tag'].values]

In [29]:
# data check - 띄어쓰기 단위로 개체명 인식 레이블이 태깅됨
print(train_data_sentence[2])
print(train_data_label[2])

['슈나이더의', '프레젠테이션은', '말', '청중을', '위한', '특별한', '쇼다', '.']
['PER-B', 'O', 'O', 'CVL-B', 'O', 'O', 'O', 'O']


In [30]:
labels = [label.strip() for label in open('ner_label.txt', 'r', encoding='utf-8')]
print('개체명 태깅 정보 :', labels)

개체명 태깅 정보 : ['O', 'PER-B', 'PER-I', 'FLD-B', 'FLD-I', 'AFW-B', 'AFW-I', 'ORG-B', 'ORG-I', 'LOC-B', 'LOC-I', 'CVL-B', 'CVL-I', 'DAT-B', 'DAT-I', 'TIM-B', 'TIM-I', 'NUM-B', 'NUM-I', 'EVT-B', 'EVT-I', 'ANM-B', 'ANM-I', 'PLT-B', 'PLT-I', 'MAT-B', 'MAT-I', 'TRM-B', 'TRM-I']


In [31]:
# 개체명 태깅 정보:정수
tag_to_index = {tag: index for index, tag in enumerate(labels)}

# 정수:개체명 태깅 정보
index_to_tag = {index: tag for index, tag in enumerate(labels)}

In [32]:
tag_size = len(tag_to_index)
print('사전의 크기 :',tag_size)

사전의 크기 : 29


In [33]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [34]:
sent = train_data_sentence[1]
label = train_data_label[1]
print('문장 :', sent)
print('레이블 :',label)
print('레이블의 정수 인코딩 :',[tag_to_index[idx] for idx in label])
print('문장의 길이 :', len(sent))
print('레이블의 길이 :', len(label))

문장 : ['▶', '쿠마리', '한동수가', '말하는', "'가넷", '&', "에르덴'"]
레이블 : ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
레이블의 정수 인코딩 : [0, 1, 2, 0, 1, 0, 1]
문장의 길이 : 7
레이블의 길이 : 7


In [35]:
tokens = []

for one_word in sent:
  # 각 단어에 대해서 서브워드로 분리.
  # ex) one_word = '쿠마리' ===> subword_tokens = ['쿠', '##마 리']
  # ex) one_word = '한동수가' ===> subword_tokens = ['한동', '##수', '##가']
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)

print('토크나이저 적용 후 문장 :',tokens)
print('레이블 :', label)
print('레이블의 정수 인코딩 :',[tag_to_index[idx] for idx in label])
print('문장의 길이 :', len(tokens))
print('레이블의 길이 :', len(label))

토크나이저 적용 후 문장 : ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레이블 : ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
레이블의 정수 인코딩 : [0, 1, 2, 0, 1, 0, 1]
문장의 길이 : 16
레이블의 길이 : 7


In [36]:
# 서브워드로 분리하게 되면서 문장의 길이가 길어짐
# 레이블과의 길이가 달라졌기 때문에 일치하도록 맞춰주어야 한다.
# 서브워드된 것 중에서 앞 쪽에 있는 것은 기존의 레이블을 붙여주고 뒤의 서브워드는 레이블하지 않기 위해 -100의 값을 준다.
# 예시: 쿠('PER-B'), ##마리(-100)
tokens = []
labels_ids = []

for one_word, label_token in zip(train_data_sentence[1], train_data_label[1]):
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)
  labels_ids.extend([tag_to_index[label_token]]+ [-100] * (len(subword_tokens) - 1))

# 서브워드 분리된 것
print('토큰화 후 문장 :',tokens)

# 레이블하지 않기 위해 -100을 붙여준 것에 [PAD] 토큰을 추가
print('레이블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in labels_ids])
print('레이블의 정수 인코딩 :', labels_ids)
print('문장의 길이 :', len(tokens))
print('레이블의 길이 :', len(labels_ids))

토큰화 후 문장 : ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레이블 : ['O', 'PER-B', '[PAD]', 'PER-I', '[PAD]', '[PAD]', 'O', '[PAD]', '[PAD]', 'PER-B', '[PAD]', '[PAD]', 'O', 'PER-B', '[PAD]', '[PAD]']
레이블의 정수 인코딩 : [0, 1, -100, 2, -100, -100, 0, -100, -100, 1, -100, -100, 0, 1, -100, -100]
문장의 길이 : 16
레이블의 길이 : 16


In [37]:
# 위 과정을 종합한 함수 지정
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
  # 토크나이저의 특수 토큰 가져오기
  cls_token = tokenizer.cls_token
  sep_token = tokenizer.sep_token
  pad_token_id = tokenizer.pad_token_id

  # 임베딩 벡터를 저장하기 위한 빈 리스트 생성
  input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

  for example, label in tqdm(zip(examples, labels), total=len(examples)):
    tokens = []
    labels_ids = []
    for one_word, label_token in zip(example, label):

    # 하나의 단어에 대해서 서브워드로 토큰화
      subword_tokens = tokenizer.tokenize(one_word)
      tokens.extend(subword_tokens)
      # 서브워드 중 첫 번째 서브워드만 개체명 레이블(기존)을 주고 그외에는 -100을 붙인다.
      labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

    # [CLS]와 [SEP]를 후에 추가할 것을 고려하여 최대 길이를 초과하는 샘플의 경우 max_seq_len - 2의 길이로 변환
    # ex) max_seq_len = 64라면 길이가 62보다 긴 샘플은 뒷 부분을 자르고 길이 62로 변환
    special_tokens_count = 2 # 감소하고자 하는 count

    if len(tokens) > max_seq_len - special_tokens_count:
      tokens = tokens[:(max_seq_len - special_tokens_count)]
      labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

    # [SEP] 추가
    # 1. 토큰화 된 결과의 뒷 부분에 [SEP]추가
    # 2. 레이블의 맨 뒷 부분에 -100 추가
    tokens += [sep_token]
    labels_ids += [pad_token_id_for_label]

    # [CLS] 추가
    # 1. 토큰화 된 결과의 앞 부분에 [CLS] 추가
    # 2. 레이블의 맨 앞 부분에 -100 추가
    tokens = [cls_token] + tokens
    labels_ids = [pad_token_id_for_label] + labels_ids

    # 정수 인코딩
    input_id = tokenizer.convert_tokens_to_ids(tokens)

    # Attention mask 생성
    attention_mask = [1] * len(input_id)

    # 패딩길이 계산
    padding_count = max_seq_len - len(input_id)

    # 계산한 패딩 길이 만큼 패딩해주기
    input_id = input_id + ([pad_token_id] * padding_count)

    # 패딩인 부분만큼 0 넣어주기
    attention_mask = attention_mask + ([0] * padding_count)

    # 세그먼트 인코딩
    # pad_token_id_for_segment=0
    token_type_id = [pad_token_id_for_segment] * max_seq_len

    # 레이블 패딩(패딩 토큰의 ID = -100)
    label = labels_ids + ([pad_token_id_for_label] * padding_count)

    assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
    assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

    # 생성한 빈 리스트에 넣기
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    data_labels.append(label)

  # 넘파이 배열로 변환
  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  data_labels = np.asarray(data_labels, dtype=np.int32)

  return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
# 훈련 데이터와 test 데이터에 수행
X_train, y_train = convert_examples_to_features(train_data_sentence,train_data_label, max_seq_len=128, tokenizer=tokenizer)
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

In [38]:
class TFBertForTokenClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForTokenClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.classifier = tf.keras.layers.Dense(num_labels,
                                            kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                            name='classifier')

  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

    # 전체 시퀀스에 대해서 분류 해야 하므로 outputs[0]임에 주의
    all_output = outputs[0]
    prediction = self.classifier(all_output)
    return prediction

-100이라고 지정한 레이블 제외시키기(test 데이터에서 서브워드 토큰화를 진행하면서 지정했었다)
- 실제값: [‑100, 2, 1, ‑100]
- 예측값: [예측값 1, 예측값 2, 예측값 3, 예측값 4]


만약 레이블이 -100이라면
- 예측값: [예측값 2, 예측값 3]

이렇게 제외된다.

In [39]:
# -100이라고 지정한 레이블 제외시키기(test 데이터에서 서브워드 토큰화를 진행하면서 지정했었다)
# 손실함수 재지정
# 목적: -100 이라는 숫자를 레이블에 대한 [PAD] 토큰으로 간주하여 loss 를 구하지 않는 것
# 기존에 했던 문제처럼 SparseCategoricalCrossentropy를 사용

def compute_loss(labels, logits):
  # 다중 클래스 분류 문제에서 softmax를 사용하지 않는다면 from_logits=True로 설정
  # softmax 함수를 통과시키지 않은 것을 감안하여 오차를 구해준다.
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(rom_logits=True, reduction=tf.keras.losses.Reduction.NONE)

  # -100의 값을 가진 정수에 대해서는 오차를 반영하지 않도록 labels를 수정
  active_loss = tf.reshape(labels, (-1,)) != -100

  # activa_loss로부터 reduced_logits과 labels를 각각 얻는다.
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
  return loss_fn(labels, reduced_logits)

### 학습

In [ ]:
class F1score(tf.keras.callbacks.Callback):
  def __init__(self, X_test, y_test):
    self.X_test = X_test
    self.y_test = y_test

  def sequences_to_tags(self, label_ids, pred_ids):
    label_list = []
    pred_list = []

    for i in range(0, len(label_ids)):
      label_tag = []
      pred_tag = []

      # 레이블이 -100인 경우는 F1 score 계산할 때도 제외
      # label_index : [1 -100 2 -100] ===> [1 2] ===> label_tag : [PER-B PER-I]
      for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
        if label_index != -100:
          label_tag.append(index_to_tag[label_index])
          pred_tag.append(index_to_tag[pred_index])

      label_list.append(label_tag)
      pred_list.append(pred_tag)

    return label_list, pred_list

    # epoch이 끝날 때마다 실행되는 함수
    def on_epoch_end(self, epoch, logs={}):
      y_predicted = self.model.predict(self.X_test)
      y_predicted = np.argmax(y_predicted, axis = 2)
      label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)
      score = f1_score(label_list, pred_list, suffix=True)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(label_list, pred_list, suffix=True))

In [ ]:
# 평가 지표: F1 score
f1_score_report = F1score(X_test, y_test)
model.fit(X_train, y_train, epochs=3, batch_size=32,callbacks = [f1_score_report])

## Faiss 와 SBERT 를 이용한 시맨틱 검색기(Semantic Search)

In [41]:
!pip install faiss-gpu
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=755e3a193d6e8e1ce5b5b26c4686f6d84b029d2186ba2d6601f2797fcdda6c42
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [43]:
import faiss
import time
from sentence_transformers import SentenceTransformer

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/19.%20Topic%20Modeling%20(LDA%2C%20BERT-Based)/dataset/abcnews-date-text.csv", filename="abcnews-date-text.csv")
df = pd.read_csv("abcnews-date-text.csv")
data = df.headline_text.to_list()

### SBERT 임베딩

In [46]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoded_data = model.encode(data)
print('임베딩 된 벡터의 수 :', len(encoded_data))

임베딩 된 벡터의 수 : 1082168


### 인덱스 정의 및 데이터 추가

In [ ]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(data))))
faiss.write_index(index, 'abc_news')

In [ ]:
def search(query):
  t = time.time()
  query_vector = model.encode([query])
  k = 5
  top_k = index.search(query_vector, k)
  print('total time: {}'.format(time.time() - t))
  return [data[_id] for _id in top_k[1].tolist()[0]]

In [ ]:
query = str(input())
results = search(query)
print('results :')
for result in results:
  print('\t', result)